# [Hands-On] 정규표현식 (Regular Expression) 실습

- Author: Sangkeun Jung (hugmanskj@gmail.com)

> 교육 목적

**Copyright**: All rights reserved

---

## Overview

이 실습에서는 정규표현식(Regular Expression)의 핵심 개념과 실제 활용법을 배웁니다.
의료 텍스트 데이터를 활용하여 실전에서 자주 사용되는 패턴 매칭 기법들을 익힐 것입니다.

실습 내용:

- 정규표현식의 기본 개념 이해
- 앵커(Anchors): 문자열의 시작과 끝
- 문자 클래스(Character Classes): 특정 문자 집합 매칭
- 수량자(Quantifiers): 반복 패턴
- 그룹핑(Grouping): 패턴 캡처 및 추출
- 선택(Alternation): 여러 패턴 중 하나 선택
- 룩어라운드(Lookaround): 맥락 기반 검사
- 플래그(Flags): 전역 옵션 설정
- 실전 예제: 의료 텍스트 파싱

정규표현식은 텍스트 처리의 스위스 아미 나이프입니다!

## 1. 정규표현식의 이론적 배경

### Stephen Kleene (1956)

"인간의 두뇌가 패턴을 인식하듯, 기계도 유한한 규칙으로 패턴을 인식할 수 있다."

**정규표현식(Regular Expression)이란?**
- 문자열 패턴을 유한한 규칙(Regular Language)으로 표현한 기호적 언어
- 기계가 '인식(accept)' 할 수 있는 입력 집합 → 정규 집합 (Regular Set)

### Regular Language 계층 구조

```
Chomsky 계층 구조에서:
- 가장 단순한 형태: Regular Language (Type 3)
- Finite-State Automaton으로 인식 가능
- 정규표현식으로 완벽히 표현 가능
```

### 왜 정규표현식을 배워야 할까?

1. **텍스트 검색**: 복잡한 패턴을 간결하게 표현
2. **데이터 검증**: 이메일, 전화번호, 날짜 형식 등 검증
3. **텍스트 추출**: 구조화된 정보 추출 (예: 의료 수치, 날짜)
4. **텍스트 변환**: 패턴 기반 치환 및 정규화
5. **로그 분석**: 대용량 로그에서 특정 패턴 찾기

---

## 2. 라이브러리 Import 및 헬퍼 함수

In [ ]:
import re
import sys
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pandas as pd

# 시각화를 위한 함수
def highlight_matches(text: str, pattern: str, flags=0) -> None:
    """
    텍스트에서 정규표현식 매칭 결과를 시각적으로 표시
    """
    matches = list(re.finditer(pattern, text, flags))

    if not matches:
        print(f"매칭 없음")
        print(f"   텍스트: {text}")
        return

    print(f"✓ {len(matches)}개 매칭 발견")
    print(f"   원본: {text}")

    # 매칭된 부분 강조 표시
    highlighted = text
    offset = 0
    for match in matches:
        start, end = match.span()
        start += offset
        end += offset
        highlighted = (
            highlighted[:start] +
            f"【{highlighted[start:end]}】" +
            highlighted[end:]
        )
        offset += 2  # 【】 추가로 인한 오프셋

    print(f"   매칭: {highlighted}")

    # 매칭 상세 정보
    for i, match in enumerate(matches, 1):
        print(f"   [{i}] 위치 {match.span()}: '{match.group()}'")
        if match.groups():
            print(f"       그룹: {match.groups()}")

def print_section(title: str) -> None:
    """섹션 제목 출력"""
    print("\n" + "=" * 70)
    print(f"  {title}")
    print("=" * 70)

def print_example(description: str) -> None:
    """예제 설명 출력"""
    print(f"\n📝 {description}")
    print("-" * 70)

print("라이브러리 및 헬퍼 함수 로드 완료!")

라이브러리 및 헬퍼 함수 로드 완료!


---

## 3. 기본 패턴: Dot (점)

In [ ]:
print_section("Dot (.) - 아무 문자 1개")

print_example("예제 1: 기본 Dot 사용")
text1 = "A1B A2B A*B"
pattern1 = r"A.B"
print(f"패턴: {pattern1}")
print(f"의미: A와 B 사이에 아무 글자 한 개 (개행 제외)")
highlight_matches(text1, pattern1)

print_example("예제 2: Dot은 개행을 매칭하지 않음")
text2 = "A1B\nA\nB"
pattern2 = r"A.B"
print(f"패턴: {pattern2}")
print(f"텍스트: A1B\\nA\\nB")
highlight_matches(text2, pattern2)

print_example("예제 3: (?s) 플래그로 개행 포함")
pattern3 = r"(?s)A.B"
print(f"패턴: {pattern3}")
print(f"의미: (?s)는 DOTALL 모드 - 점이 개행까지 매칭")
highlight_matches(text2, pattern3)


  Dot (.) - 아무 문자 1개

📝 예제 1: 기본 Dot 사용
----------------------------------------------------------------------
패턴: A.B
의미: A와 B 사이에 아무 글자 한 개 (개행 제외)
✓ 3개 매칭 발견
   원본: A1B A2B A*B
   매칭: 【A1B】 【A2B】 【A*B】
   [1] 위치 (0, 3): 'A1B'
   [2] 위치 (4, 7): 'A2B'
   [3] 위치 (8, 11): 'A*B'

📝 예제 2: Dot은 개행을 매칭하지 않음
----------------------------------------------------------------------
패턴: A.B
텍스트: A1B\nA\nB
✓ 1개 매칭 발견
   원본: A1B
A
B
   매칭: 【A1B】
A
B
   [1] 위치 (0, 3): 'A1B'

📝 예제 3: (?s) 플래그로 개행 포함
----------------------------------------------------------------------
패턴: (?s)A.B
의미: (?s)는 DOTALL 모드 - 점이 개행까지 매칭
✓ 2개 매칭 발견
   원본: A1B
A
B
   매칭: 【A1B】
【A
B】
   [1] 위치 (0, 3): 'A1B'
   [2] 위치 (4, 7): 'A
B'


---

## 4. 앵커(Anchors): ^ 와 $

In [ ]:
print_section("Anchors (^, $) - 위치 지정")

print_example("예제 1: ^ (줄의 시작)")
text4 = "BP 120/80 mmHg\nHeart Rate: 78 bpm"
pattern4 = r"^BP"
print(f"패턴: {pattern4}")
print(f"의미: 줄의 시작에서 'BP' 찾기")
highlight_matches(text4, pattern4)

print_example("예제 2: $ (줄의 끝)")
pattern5 = r"mmHg$"
print(f"패턴: {pattern5}")
print(f"의미: 줄의 끝에 있는 'mmHg' 찾기")
highlight_matches(text4, pattern5)

print_example("예제 3: 멀티라인 모드 (?m)")
text6 = "BP 120/80\nBP 130/85"
pattern6 = r"(?m)^BP"
print(f"패턴: {pattern6}")
print(f"의미: (?m) 멀티라인 모드 - 각 줄의 시작마다 매칭")
highlight_matches(text6, pattern6)


  Anchors (^, $) - 위치 지정

📝 예제 1: ^ (줄의 시작)
----------------------------------------------------------------------
패턴: ^BP
의미: 줄의 시작에서 'BP' 찾기
✓ 1개 매칭 발견
   원본: BP 120/80 mmHg
Heart Rate: 78 bpm
   매칭: 【BP】 120/80 mmHg
Heart Rate: 78 bpm
   [1] 위치 (0, 2): 'BP'

📝 예제 2: $ (줄의 끝)
----------------------------------------------------------------------
패턴: mmHg$
의미: 줄의 끝에 있는 'mmHg' 찾기
매칭 없음
   텍스트: BP 120/80 mmHg
Heart Rate: 78 bpm

📝 예제 3: 멀티라인 모드 (?m)
----------------------------------------------------------------------
패턴: (?m)^BP
의미: (?m) 멀티라인 모드 - 각 줄의 시작마다 매칭
✓ 2개 매칭 발견
   원본: BP 120/80
BP 130/85
   매칭: 【BP】 120/80
【BP】 130/85
   [1] 위치 (0, 2): 'BP'
   [2] 위치 (10, 12): 'BP'


---

## 5. 문자 클래스(Character Classes)

In [ ]:
print_section("Character Classes ([...]) - 허용 문자 집합")

print_example("예제 1: 기본 문자 클래스")
text7 = "BP 120/80 mmHg"
pattern7 = r"[A-Z]+"
print(f"패턴: {pattern7}")
print(f"의미: 대문자 연속 (한 개 이상)")
highlight_matches(text7, pattern7)

print_example("예제 2: 숫자 클래스 (\\d)")
pattern8 = r"\d+"
print(f"패턴: {pattern8}")
print(f"의미: \\d는 [0-9]와 동일 - 숫자 한 개 이상")
highlight_matches(text7, pattern8)

print_example("예제 3: 부정 문자 클래스 ([^...])")
pattern9 = r"[^0-9]+"
print(f"패턴: {pattern9}")
print(f"의미: [^0-9]는 숫자가 아닌 문자들")
highlight_matches(text7, pattern9)

print_example("예제 4: 단축 문자 클래스")
text10 = "test_123 data-456"
patterns = {
    r"\w+": "단어 문자 (알파벳, 숫자, 밑줄)",
    r"\s+": "공백 문자 (스페이스, 탭, 개행)",
    r"\d+": "숫자",
}
for pattern, desc in patterns.items():
    print(f"\n패턴: {pattern} - {desc}")
    highlight_matches(text10, pattern)


  Character Classes ([...]) - 허용 문자 집합

📝 예제 1: 기본 문자 클래스
----------------------------------------------------------------------
패턴: [A-Z]+
의미: 대문자 연속 (한 개 이상)
✓ 2개 매칭 발견
   원본: BP 120/80 mmHg
   매칭: 【BP】 120/80 mm【H】g
   [1] 위치 (0, 2): 'BP'
   [2] 위치 (12, 13): 'H'

📝 예제 2: 숫자 클래스 (\d)
----------------------------------------------------------------------
패턴: \d+
의미: \d는 [0-9]와 동일 - 숫자 한 개 이상
✓ 2개 매칭 발견
   원본: BP 120/80 mmHg
   매칭: BP 【120】/【80】 mmHg
   [1] 위치 (3, 6): '120'
   [2] 위치 (7, 9): '80'

📝 예제 3: 부정 문자 클래스 ([^...])
----------------------------------------------------------------------
패턴: [^0-9]+
의미: [^0-9]는 숫자가 아닌 문자들
✓ 3개 매칭 발견
   원본: BP 120/80 mmHg
   매칭: 【BP 】120【/】80【 mmHg】
   [1] 위치 (0, 3): 'BP '
   [2] 위치 (6, 7): '/'
   [3] 위치 (9, 14): ' mmHg'

📝 예제 4: 단축 문자 클래스
----------------------------------------------------------------------

패턴: \w+ - 단어 문자 (알파벳, 숫자, 밑줄)
✓ 3개 매칭 발견
   원본: test_123 data-456
   매칭: 【test_123】 【data】-【456】
   [1] 위치 (0, 8): 'test_123'
   [2] 위치 (9

---

## 6. 수량자(Quantifiers)

In [ ]:
print_section("Quantifiers (*, +, ?, {m,n}) - 반복 횟수")

print_example("예제 1: * (0회 이상)")
text11 = "baaaad apple bpple"
pattern11 = r"ba*"
print(f"패턴: {pattern11}")
print(f"의미: b 다음에 a가 0회 이상")
highlight_matches(text11, pattern11)

print_example("예제 2: + (1회 이상)")
pattern12 = r"ba+"
print(f"패턴: {pattern12}")
print(f"의미: b 다음에 a가 1회 이상")
highlight_matches(text11, pattern12)

print_example("예제 3: ? (0회 또는 1회)")
text13 = "color colour"
pattern13 = r"colou?r"
print(f"패턴: {pattern13}")
print(f"의미: u가 있어도 되고 없어도 됨")
highlight_matches(text13, pattern13)

print_example("예제 4: {m,n} (m회 이상 n회 이하)")
text14 = "BP 120/80 mmHg"
pattern14 = r"\d{2,3}"
print(f"패턴: {pattern14}")
print(f"의미: 숫자 2~3자리")
highlight_matches(text14, pattern14)

print_example("예제 5: 탐욕적(Greedy) vs 게으른(Lazy) 매칭")
text15 = "<div>content1</div><div>content2</div>"

print("\n탐욕적 매칭 (기본):")
pattern15a = r"<div>.*</div>"
print(f"패턴: {pattern15a}")
print(f"의미: 가능한 한 많이 매칭 (마지막 </div>까지)")
highlight_matches(text15, pattern15a)

print("\n게으른 매칭 (*?):")
pattern15b = r"<div>.*?</div>"
print(f"패턴: {pattern15b}")
print(f"의미: 가능한 한 적게 매칭 (첫 번째 </div>까지)")
highlight_matches(text15, pattern15b)


  Quantifiers (*, +, ?, {m,n}) - 반복 횟수

📝 예제 1: * (0회 이상)
----------------------------------------------------------------------
패턴: ba*
의미: b 다음에 a가 0회 이상
✓ 2개 매칭 발견
   원본: baaaad apple bpple
   매칭: 【baaaa】d apple 【b】pple
   [1] 위치 (0, 5): 'baaaa'
   [2] 위치 (13, 14): 'b'

📝 예제 2: + (1회 이상)
----------------------------------------------------------------------
패턴: ba+
의미: b 다음에 a가 1회 이상
✓ 1개 매칭 발견
   원본: baaaad apple bpple
   매칭: 【baaaa】d apple bpple
   [1] 위치 (0, 5): 'baaaa'

📝 예제 3: ? (0회 또는 1회)
----------------------------------------------------------------------
패턴: colou?r
의미: u가 있어도 되고 없어도 됨
✓ 2개 매칭 발견
   원본: color colour
   매칭: 【color】 【colour】
   [1] 위치 (0, 5): 'color'
   [2] 위치 (6, 12): 'colour'

📝 예제 4: {m,n} (m회 이상 n회 이하)
----------------------------------------------------------------------
패턴: \d{2,3}
의미: 숫자 2~3자리
✓ 2개 매칭 발견
   원본: BP 120/80 mmHg
   매칭: BP 【120】/【80】 mmHg
   [1] 위치 (3, 6): '120'
   [2] 위치 (7, 9): '80'

📝 예제 5: 탐욕적(Greedy) vs 게으른(Lazy) 매칭
----------------

---

## 7. 그룹핑(Grouping)

In [ ]:
print_section("Grouping ((...)) - 캡처 및 그룹화")

print_example("예제 1: 기본 그룹 캡처")
text16 = "BP 120/80 mmHg"
pattern16 = r"(\d{2,3})/(\d{2,3})"
print(f"패턴: {pattern16}")
print(f"의미: 두 개의 숫자 그룹을 캡처 (수축기/이완기)")

matches = re.search(pattern16, text16)
if matches:
    print(f"✓ 매칭 발견: {matches.group()}")
    print(f"   그룹 1 (수축기): {matches.group(1)}")
    print(f"   그룹 2 (이완기): {matches.group(2)}")

print_example("예제 2: 이름있는 그룹 (?P<name>)")
pattern17 = r"(?P<systolic>\d{2,3})/(?P<diastolic>\d{2,3})"
print(f"패턴: {pattern17}")
print(f"의미: 이름이 지정된 그룹으로 캡처")

matches = re.search(pattern17, text16)
if matches:
    print(f"✓ 매칭 발견: {matches.group()}")
    print(f"   수축기 (systolic): {matches.group('systolic')}")
    print(f"   이완기 (diastolic): {matches.group('diastolic')}")
    print(f"   딕셔너리 형태: {matches.groupdict()}")

print_example("예제 3: 비캡처 그룹 (?:...)")
text18 = "CT12 MRI88 PET01"
pattern18a = r"(CT|MRI|PET)(\d+)"
pattern18b = r"(?:CT|MRI|PET)(\d+)"

print(f"캡처 그룹: {pattern18a}")
matches_a = list(re.finditer(pattern18a, text18))
print(f"매칭: {[m.group() for m in matches_a]}")
print(f"그룹 수: {[len(m.groups()) for m in matches_a]}")

print(f"\n비캡처 그룹: {pattern18b}")
matches_b = list(re.finditer(pattern18b, text18))
print(f"매칭: {[m.group() for m in matches_b]}")
print(f"그룹 수: {[len(m.groups()) for m in matches_b]}")


  Grouping ((...)) - 캡처 및 그룹화

📝 예제 1: 기본 그룹 캡처
----------------------------------------------------------------------
패턴: (\d{2,3})/(\d{2,3})
의미: 두 개의 숫자 그룹을 캡처 (수축기/이완기)
✓ 매칭 발견: 120/80
   그룹 1 (수축기): 120
   그룹 2 (이완기): 80

📝 예제 2: 이름있는 그룹 (?P<name>)
----------------------------------------------------------------------
패턴: (?P<systolic>\d{2,3})/(?P<diastolic>\d{2,3})
의미: 이름이 지정된 그룹으로 캡처
✓ 매칭 발견: 120/80
   수축기 (systolic): 120
   이완기 (diastolic): 80
   딕셔너리 형태: {'systolic': '120', 'diastolic': '80'}

📝 예제 3: 비캡처 그룹 (?:...)
----------------------------------------------------------------------
캡처 그룹: (CT|MRI|PET)(\d+)
매칭: ['CT12', 'MRI88', 'PET01']
그룹 수: [2, 2, 2]

비캡처 그룹: (?:CT|MRI|PET)(\d+)
매칭: ['CT12', 'MRI88', 'PET01']
그룹 수: [1, 1, 1]


---

## 8. 선택(Alternation)

In [ ]:
print_section("Alternation (|) - 여러 패턴 중 선택")

print_example("예제 1: 기본 선택")
text19 = "CT12 MRI88 PET01"
pattern19 = r"CT|MRI"
print(f"패턴: {pattern19}")
print(f"의미: CT 또는 MRI")
highlight_matches(text19, pattern19)

print_example("예제 2: 그룹과 함께 사용")
pattern20 = r"(CT|MRI|PET)\d+"
print(f"패턴: {pattern20}")
print(f"의미: CT, MRI, 또는 PET 뒤에 숫자")
highlight_matches(text19, pattern20)

print_example("예제 3: 복잡한 패턴 선택")
text21 = "혈압: 120/80, BP: 130/85, blood pressure: 140/90"
pattern21 = r"(혈압|BP|blood pressure):\s*(\d+)/(\d+)"
print(f"패턴: {pattern21}")
print(f"의미: 다양한 표현 + 숫자 패턴")

matches = list(re.finditer(pattern21, text21))
for i, match in enumerate(matches, 1):
    print(f"[{i}] {match.group()} - 표현: {match.group(1)}, 수치: {match.group(2)}/{match.group(3)}")


  Alternation (|) - 여러 패턴 중 선택

📝 예제 1: 기본 선택
----------------------------------------------------------------------
패턴: CT|MRI
의미: CT 또는 MRI
✓ 2개 매칭 발견
   원본: CT12 MRI88 PET01
   매칭: 【CT】12 【MRI】88 PET01
   [1] 위치 (0, 2): 'CT'
   [2] 위치 (5, 8): 'MRI'

📝 예제 2: 그룹과 함께 사용
----------------------------------------------------------------------
패턴: (CT|MRI|PET)\d+
의미: CT, MRI, 또는 PET 뒤에 숫자
✓ 3개 매칭 발견
   원본: CT12 MRI88 PET01
   매칭: 【CT12】 【MRI88】 【PET01】
   [1] 위치 (0, 4): 'CT12'
       그룹: ('CT',)
   [2] 위치 (5, 10): 'MRI88'
       그룹: ('MRI',)
   [3] 위치 (11, 16): 'PET01'
       그룹: ('PET',)

📝 예제 3: 복잡한 패턴 선택
----------------------------------------------------------------------
패턴: (혈압|BP|blood pressure):\s*(\d+)/(\d+)
의미: 다양한 표현 + 숫자 패턴
[1] 혈압: 120/80 - 표현: 혈압, 수치: 120/80
[2] BP: 130/85 - 표현: BP, 수치: 130/85
[3] blood pressure: 140/90 - 표현: blood pressure, 수치: 140/90


---

## 9. 룩어라운드(Lookaround)

In [ ]:
print_section("Lookaround - 맥락 기반 검사 (문자 소비 없음)")

print_example("예제 1: Positive Lookahead (?=...)")
text22 = "BP120 mmHg, BP high"
pattern22 = r"BP(?=\d+)"
print(f"패턴: {pattern22}")
print(f"의미: 뒤에 숫자가 있는 'BP'만 매칭")
highlight_matches(text22, pattern22)

print_example("예제 2: Negative Lookahead (?!...)")
pattern23 = r"BP(?!\d+)"
print(f"패턴: {pattern23}")
print(f"의미: 뒤에 숫자가 없는 'BP'만 매칭")
highlight_matches(text22, pattern23)

print_example("예제 3: Positive Lookbehind (?<=...)")
text24 = "price: 100원, 200달러, 300엔"
pattern24 = r"(?<=price: )\d+"
print(f"패턴: {pattern24}")
print(f"의미: 'price: ' 바로 뒤의 숫자")
highlight_matches(text24, pattern24)

print_example("예제 4: Negative Lookbehind (?<!...)")
text25 = "CT: 150, MRI: 200, X-ray: 100"
pattern25 = r"(?<!CT: )\d+"
print(f"패턴: {pattern25}")
print(f"의미: 'CT: ' 뒤가 아닌 숫자들")
highlight_matches(text25, pattern25)

print_example("예제 5: 복합 룩어라운드 - 숫자 사이의 단위 추출")
text26 = "BP 120/80 mmHg, 체온 36.5°C, 맥박 72회/분"
pattern26 = r"(?<=\d\s)[\w°/]+(?=,|\s|$)"
print(f"패턴: {pattern26}")
print(f"의미: 숫자 뒤, 구분자 앞의 단위")
highlight_matches(text26, pattern26)


  Lookaround - 맥락 기반 검사 (문자 소비 없음)

📝 예제 1: Positive Lookahead (?=...)
----------------------------------------------------------------------
패턴: BP(?=\d+)
의미: 뒤에 숫자가 있는 'BP'만 매칭
✓ 1개 매칭 발견
   원본: BP120 mmHg, BP high
   매칭: 【BP】120 mmHg, BP high
   [1] 위치 (0, 2): 'BP'

📝 예제 2: Negative Lookahead (?!...)
----------------------------------------------------------------------
패턴: BP(?!\d+)
의미: 뒤에 숫자가 없는 'BP'만 매칭
✓ 1개 매칭 발견
   원본: BP120 mmHg, BP high
   매칭: BP120 mmHg, 【BP】 high
   [1] 위치 (12, 14): 'BP'

📝 예제 3: Positive Lookbehind (?<=...)
----------------------------------------------------------------------
패턴: (?<=price: )\d+
의미: 'price: ' 바로 뒤의 숫자
✓ 1개 매칭 발견
   원본: price: 100원, 200달러, 300엔
   매칭: price: 【100】원, 200달러, 300엔
   [1] 위치 (7, 10): '100'

📝 예제 4: Negative Lookbehind (?<!...)
----------------------------------------------------------------------
패턴: (?<!CT: )\d+
의미: 'CT: ' 뒤가 아닌 숫자들
✓ 3개 매칭 발견
   원본: CT: 150, MRI: 200, X-ray: 100
   매칭: CT: 1【50】, MRI: 【200】, X-ray: 【100】
  

---

## 10. 플래그(Flags)

In [ ]:
print_section("Flags - 정규표현식 전역 옵션")

print_example("예제 1: (?i) - 대소문자 무시")
text27 = "Blood pressure: 120/80\nBLOOD PRESSURE: 130/85"
pattern27 = r"(?i)blood pressure"
print(f"패턴: {pattern27}")
print(f"의미: 대소문자 구분 없이 매칭")
highlight_matches(text27, pattern27)

print_example("예제 2: (?m) - 멀티라인 모드")
text28 = "line1\nline2\nline3"
pattern28 = r"(?m)^line"
print(f"패턴: {pattern28}")
print(f"의미: 각 줄의 시작에서 'line' 매칭")
highlight_matches(text28, pattern28)

print_example("예제 3: (?s) - DOTALL 모드")
text29 = "start\nmiddle\nend"
pattern29 = r"(?s)start.*end"
print(f"패턴: {pattern29}")
print(f"의미: .이 개행 문자도 포함")
highlight_matches(text29, pattern29)

print_example("예제 4: (?x) - VERBOSE 모드 (가독성)")
text30 = "2025-04-16"
# 가독성 좋은 패턴 작성
pattern30 = r"""(?x)
    (\d{4})  # 년도 4자리
    -        # 구분자
    (\d{2})  # 월 2자리
    -        # 구분자
    (\d{2})  # 일 2자리
"""
print(f"패턴: (VERBOSE 모드 - 공백과 주석 무시)")
print(pattern30)

matches = re.search(pattern30, text30)
if matches:
    print(f"매칭 발견: {matches.group()}")
    print(f"   년: {matches.group(1)}, 월: {matches.group(2)}, 일: {matches.group(3)}")


  Flags - 정규표현식 전역 옵션

📝 예제 1: (?i) - 대소문자 무시
----------------------------------------------------------------------
패턴: (?i)blood pressure
의미: 대소문자 구분 없이 매칭
✓ 2개 매칭 발견
   원본: Blood pressure: 120/80
BLOOD PRESSURE: 130/85
   매칭: 【Blood pressure】: 120/80
【BLOOD PRESSURE】: 130/85
   [1] 위치 (0, 14): 'Blood pressure'
   [2] 위치 (23, 37): 'BLOOD PRESSURE'

📝 예제 2: (?m) - 멀티라인 모드
----------------------------------------------------------------------
패턴: (?m)^line
의미: 각 줄의 시작에서 'line' 매칭
✓ 3개 매칭 발견
   원본: line1
line2
line3
   매칭: 【line】1
【line】2
【line】3
   [1] 위치 (0, 4): 'line'
   [2] 위치 (6, 10): 'line'
   [3] 위치 (12, 16): 'line'

📝 예제 3: (?s) - DOTALL 모드
----------------------------------------------------------------------
패턴: (?s)start.*end
의미: .이 개행 문자도 포함
✓ 1개 매칭 발견
   원본: start
middle
end
   매칭: 【start
middle
end】
   [1] 위치 (0, 16): 'start
middle
end'

📝 예제 4: (?x) - VERBOSE 모드 (가독성)
----------------------------------------------------------------------
패턴: (VERBOSE 모드 - 공백과 주석 무시)
(?x)

---

## 11. 실전 예제 1: 의료 텍스트 파싱

In [ ]:
print_section("실전 예제 1: 의료 바이탈 사인 추출")

# 샘플 의료 텍스트
medical_text = """
환자명: 홍길동
검사일: 2025-04-16
바이탈 사인:
- BP: 120/80 mmHg
- 체온: 36.5°C
- 맥박: 72 bpm
- 호흡: 18회/분
- SpO2: 98%

검사 결과:
- CT: 정상 소견
- MRI: 이상 없음
- X-ray: 정상
혈액검사: WBC 7.2, RBC 4.5, Hb 14.2 g/dL
"""

print("📋 원본 텍스트:")
print(medical_text)

print_example("1. 혈압 추출")
bp_pattern = r"BP:\s*(?P<systolic>\d{2,3})/(?P<diastolic>\d{2,3})\s*mmHg"
bp_match = re.search(bp_pattern, medical_text)
if bp_match:
    print(f"✓ 수축기: {bp_match.group('systolic')} mmHg")
    print(f"✓ 이완기: {bp_match.group('diastolic')} mmHg")

print_example("2. 모든 수치 + 단위 추출")
vital_pattern = r"(\w+):\s*(\d+(?:\.\d+)?)\s*([°%\w/]+)"
vital_matches = re.findall(vital_pattern, medical_text)
print("✓ 발견된 바이탈 사인:")
for item, value, unit in vital_matches:
    print(f"   - {item}: {value} {unit}")

print_example("3. 날짜 추출 및 검증")
date_pattern = r"(\d{4})-(\d{2})-(\d{2})"
date_match = re.search(date_pattern, medical_text)
if date_match:
    year, month, day = date_match.groups()
    print(f"✓ 검사일: {year}년 {month}월 {day}일")

print_example("4. 검사 종류 추출")
exam_pattern = r"(CT|MRI|X-ray):\s*([^\n]+)"
exam_matches = re.findall(exam_pattern, medical_text)
print("✓ 영상 검사 결과:")
for exam_type, result in exam_matches:
    print(f"   - {exam_type}: {result}")

print_example("5. 혈액검사 수치 추출")
blood_pattern = r"(\w+)\s+(\d+(?:\.\d+)?)"
# 혈액검사 섹션만 추출
blood_section = re.search(r"혈액검사:([^\n]+)", medical_text)
if blood_section:
    blood_matches = re.findall(blood_pattern, blood_section.group(1))
    print("혈액검사 수치:")
    for test, value in blood_matches:
        print(f"   - {test}: {value}")


  실전 예제 1: 의료 바이탈 사인 추출
📋 원본 텍스트:

환자명: 홍길동
검사일: 2025-04-16
바이탈 사인:
- BP: 120/80 mmHg
- 체온: 36.5°C
- 맥박: 72 bpm
- 호흡: 18회/분
- SpO2: 98%

검사 결과:
- CT: 정상 소견
- MRI: 이상 없음
- X-ray: 정상
혈액검사: WBC 7.2, RBC 4.5, Hb 14.2 g/dL


📝 1. 혈압 추출
----------------------------------------------------------------------
✓ 수축기: 120 mmHg
✓ 이완기: 80 mmHg

📝 2. 모든 수치 + 단위 추출
----------------------------------------------------------------------
✓ 발견된 바이탈 사인:
   - 검사일: 202 5
   - BP: 120 /80
   - 체온: 36.5 °C
   - 맥박: 72 bpm
   - 호흡: 18 회/분
   - SpO2: 98 %

📝 3. 날짜 추출 및 검증
----------------------------------------------------------------------
✓ 검사일: 2025년 04월 16일

📝 4. 검사 종류 추출
----------------------------------------------------------------------
✓ 영상 검사 결과:
   - CT: 정상 소견
   - MRI: 이상 없음
   - X-ray: 정상

📝 5. 혈액검사 수치 추출
----------------------------------------------------------------------
혈액검사 수치:
   - WBC: 7.2
   - RBC: 4.5
   - Hb: 14.2


---

## 12. 실전 예제 2: 데이터 검증

In [ ]:
print_section("실전 예제 2: 데이터 검증 및 정규화")

print_example("1. 전화번호 검증")
phone_samples = [
    "010-1234-5678",
    "02-123-4567",
    "031-1234-5678",
    "010 1234 5678",
    "01012345678",
    "123-456-7890",  # 잘못된 형식
]

phone_pattern = r"^0\d{1,2}-?\d{3,4}-?\d{4}$"
print("전화번호 패턴:", phone_pattern)
print("\n검증 결과:")
for phone in phone_samples:
    is_valid = bool(re.match(phone_pattern, phone))
    status = "✓ 유효" if is_valid else "✗ 무효"
    print(f"{status}: {phone}")

print_example("2. 이메일 검증 (간단한 버전)")
email_samples = [
    "user@example.com",
    "test.user@domain.co.kr",
    "invalid@",
    "@invalid.com",
    "no-at-sign.com",
]

email_pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
print("이메일 패턴:", email_pattern)
print("\n검증 결과:")
for email in email_samples:
    is_valid = bool(re.match(email_pattern, email))
    status = "✓ 유효" if is_valid else "✗ 무효"
    print(f"{status}: {email}")

print_example("3. 날짜 형식 변환")
date_samples = [
    "2025-04-16",
    "2025/04/16",
    "04-16-2025",
    "16/04/2025",
]

# YYYY-MM-DD 형식 찾기
iso_pattern = r"(\d{4})[-/](\d{2})[-/](\d{2})"
# MM-DD-YYYY 형식 찾기
us_pattern = r"(\d{2})[-/](\d{2})[-/](\d{4})"

print("날짜 정규화 결과 (YYYY-MM-DD):")
for date_str in date_samples:
    # ISO 형식 시도
    match = re.match(iso_pattern, date_str)
    if match:
        print(f"{date_str} → {match.group(1)}-{match.group(2)}-{match.group(3)}")
        continue

    # US 형식 시도
    match = re.match(us_pattern, date_str)
    if match:
        print(f"{date_str} → {match.group(3)}-{match.group(1)}-{match.group(2)}")

print_example("4. 텍스트 정규화 - 공백 제거")
messy_text = "  여러    공백이    있는     텍스트  "
normalized = re.sub(r'\s+', ' ', messy_text.strip())
print(f"원본: '{messy_text}'")
print(f"정규화: '{normalized}'")


  실전 예제 2: 데이터 검증 및 정규화

📝 1. 전화번호 검증
----------------------------------------------------------------------
전화번호 패턴: ^0\d{1,2}-?\d{3,4}-?\d{4}$

검증 결과:
✓ 유효: 010-1234-5678
✓ 유효: 02-123-4567
✓ 유효: 031-1234-5678
✗ 무효: 010 1234 5678
✓ 유효: 01012345678
✗ 무효: 123-456-7890

📝 2. 이메일 검증 (간단한 버전)
----------------------------------------------------------------------
이메일 패턴: ^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$

검증 결과:
✓ 유효: user@example.com
✓ 유효: test.user@domain.co.kr
✗ 무효: invalid@
✗ 무효: @invalid.com
✗ 무효: no-at-sign.com

📝 3. 날짜 형식 변환
----------------------------------------------------------------------
날짜 정규화 결과 (YYYY-MM-DD):
2025-04-16 → 2025-04-16
2025/04/16 → 2025-04-16
04-16-2025 → 2025-04-16
16/04/2025 → 2025-16-04

📝 4. 텍스트 정규화 - 공백 제거
----------------------------------------------------------------------
원본: '  여러    공백이    있는     텍스트  '
정규화: '여러 공백이 있는 텍스트'


---

## 13. 실전 예제 3: 텍스트 추출 및 변환

In [ ]:
print_section("실전 예제 3: 의료 판독문에서 정보 추출")

# 복잡한 의료 판독문
radiology_report = """
[흉부 X-ray 판독]
검사일시: 2025-04-16 14:30
판독의: 김영상

소견:
1. 양측 폐야의 음영 증가 소견 없음.
2. 심비대 소견 없음 (CTR: 45%).
3. 종격동 내 이상 음영 없음.
4. 늑막 삼출 소견 없음.
5. 골격계: 이상 소견 없음.

결론: 정상 소견 (Normal chest PA)

이전 검사와 비교:
- 2025-03-15: 특이 변화 없음
- 2024-12-01: 경미한 폐음영 관찰됨 → 호전

권고사항:
- 6개월 후 추적검사 권장
- 증상 발생 시 즉시 내원
"""

print("📋 원본 판독문:")
print(radiology_report)

print_example("1. 구조화된 정보 추출")

# 데이터 구조 정의
@dataclass
class RadiologyReport:
    exam_type: str
    exam_date: str
    exam_time: str
    doctor: str
    ctr_value: Optional[float]
    findings: List[str]
    conclusion: str
    recommendations: List[str]

# 정보 추출
report = RadiologyReport(
    exam_type=re.search(r'\[(.*?)\]', radiology_report).group(1) if re.search(r'\[(.*?)\]', radiology_report) else None,
    exam_date=re.search(r'(\d{4}-\d{2}-\d{2})', radiology_report).group(1) if re.search(r'(\d{4}-\d{2}-\d{2})', radiology_report) else None,
    exam_time=re.search(r'\d{4}-\d{2}-\d{2}\s+(\d{2}:\d{2})', radiology_report).group(1) if re.search(r'\d{4}-\d{2}-\d{2}\s+(\d{2}:\d{2})', radiology_report) else None,
    doctor=re.search(r'판독의:\s*(.+)', radiology_report).group(1) if re.search(r'판독의:\s*(.+)', radiology_report) else None,
    ctr_value=float(re.search(r'CTR:\s*(\d+)%', radiology_report).group(1)) if re.search(r'CTR:\s*(\d+)%', radiology_report) else None,
    findings=re.findall(r'\d+\.\s+([^.]+\.)', radiology_report),
    conclusion=re.search(r'결론:\s*(.+)', radiology_report).group(1) if re.search(r'결론:\s*(.+)', radiology_report) else None,
    recommendations=re.findall(r'-\s+([^-\n]+(?:권장|내원))', radiology_report),
)

print("✓ 추출된 정보:")
print(f"   검사 종류: {report.exam_type}")
print(f"   검사 일시: {report.exam_date} {report.exam_time}")
print(f"   판독의: {report.doctor}")
print(f"   CTR 수치: {report.ctr_value}%")
print(f"   소견 개수: {len(report.findings)}개")
print(f"   결론: {report.conclusion}")
print(f"   권고사항: {len(report.recommendations)}개")

print_example("2. 이전 검사 날짜 추출")
prev_exam_pattern = r'-\s*(\d{4}-\d{2}-\d{2}):\s*([^\n]+)'
prev_exams = re.findall(prev_exam_pattern, radiology_report)
print("✓ 이전 검사 이력:")
for date, finding in prev_exams:
    print(f"   {date}: {finding}")

print_example("3. 백분율 수치 추출")
percentage_pattern = r'(\w+):\s*(\d+)%'
percentages = re.findall(percentage_pattern, radiology_report)
print("✓ 백분율 수치:")
for name, value in percentages:
    print(f"   {name}: {value}%")

print_example("4. 부정 표현 탐지")
negative_pattern = r'(\w+)\s+(없음|음성)'
negatives = re.findall(negative_pattern, radiology_report)
print("✓ 정상(부정) 소견:")
for item, _ in negatives:
    print(f"   - {item}: 정상")


  실전 예제 3: 의료 판독문에서 정보 추출
📋 원본 판독문:

[흉부 X-ray 판독]
검사일시: 2025-04-16 14:30
판독의: 김영상

소견:
1. 양측 폐야의 음영 증가 소견 없음.
2. 심비대 소견 없음 (CTR: 45%).
3. 종격동 내 이상 음영 없음.
4. 늑막 삼출 소견 없음.
5. 골격계: 이상 소견 없음.

결론: 정상 소견 (Normal chest PA)

이전 검사와 비교:
- 2025-03-15: 특이 변화 없음
- 2024-12-01: 경미한 폐음영 관찰됨 → 호전

권고사항:
- 6개월 후 추적검사 권장
- 증상 발생 시 즉시 내원


📝 1. 구조화된 정보 추출
----------------------------------------------------------------------
✓ 추출된 정보:
   검사 종류: 흉부 X-ray 판독
   검사 일시: 2025-04-16 14:30
   판독의: 김영상
   CTR 수치: 45.0%
   소견 개수: 5개
   결론: 정상 소견 (Normal chest PA)
   권고사항: 2개

📝 2. 이전 검사 날짜 추출
----------------------------------------------------------------------
✓ 이전 검사 이력:
   2025-03-15: 특이 변화 없음
   2024-12-01: 경미한 폐음영 관찰됨 → 호전

📝 3. 백분율 수치 추출
----------------------------------------------------------------------
✓ 백분율 수치:
   CTR: 45%

📝 4. 부정 표현 탐지
----------------------------------------------------------------------
✓ 정상(부정) 소견:
   - 소견: 정상
   - 소견: 정상
   - 음영: 정상
   - 소견: 정상
   - 소견: 정상
   - 변화: 정상


---

## 14. 실전 예제 4: 텍스트 클렌징 및 전처리

In [ ]:
print_section("실전 예제 4: 텍스트 전처리 파이프라인")

print_example("원본 텍스트")
raw_text = """
환자ID: 2025-001234
주민번호: 850315-1******
연락처: 010-1234-5678
이메일: patient@hospital.com

특이사항:
- 고혈압   있음  (BP:  150/95)
- 당뇨    있음 (HbA1c: 7.2%)
- 알레르기: 페니실린, 아스피린

메모:  정기검진     필요!!!

URL: https://hospital.com/records/12345
"""

print(raw_text)

print_example("1. 개인정보 마스킹")
def mask_personal_info(text):
    # 주민번호 마스킹
    text = re.sub(r'(\d{6})-([1-4])\d{6}', r'\1-\2******', text)
    # 전화번호 마스킹
    text = re.sub(r'(01\d)-(\d{4})-\d{4}', r'\1-\2-****', text)
    # 이메일 마스킹
    text = re.sub(r'([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+\.[a-zA-Z]{2,})',
                  lambda m: f"{m.group(1)[:2]}***@{m.group(2)}", text)
    return text

masked_text = mask_personal_info(raw_text)
print("마스킹 결과:")
print(masked_text)

print_example("2. 공백 정규화")
def normalize_whitespace(text):
    # 여러 공백을 하나로
    text = re.sub(r'[ \t]+', ' ', text)
    # 여러 줄바꿈을 최대 2개로
    text = re.sub(r'\n{3,}', '\n\n', text)
    # 줄 앞뒤 공백 제거
    text = re.sub(r'(?m)^[ \t]+|[ \t]+$', '', text)
    return text

normalized_text = normalize_whitespace(masked_text)
print("공백 정규화 결과:")
print(repr(normalized_text[:100]))  # 앞부분만 표시

print_example("3. URL 추출 및 제거")
url_pattern = r'https?://[^\s]+'
urls = re.findall(url_pattern, normalized_text)
print(f"✓ 발견된 URL: {urls}")

text_without_urls = re.sub(url_pattern, '[URL 제거됨]', normalized_text)
print("URL 제거 후:")
print(text_without_urls[:200])

print_example("4. 숫자와 단위 정규화")
def normalize_measurements(text):
    # 혈압 정규화
    text = re.sub(r'BP\s*:\s*(\d+)\s*/\s*(\d+)', r'혈압=\1/\2mmHg', text)
    # 백분율 정규화
    text = re.sub(r'(\d+(?:\.\d+)?)\s*%', r'\1%', text)
    return text

normalized_measurements = normalize_measurements(text_without_urls)
print("측정값 정규화 결과:")
print(normalized_measurements[:300])

print_example("5. 특수문자 정리")
def clean_special_chars(text):
    # 느낌표 3개 이상을 1개로
    text = re.sub(r'!{2,}', '!', text)
    # 불필요한 특수문자 제거
    text = re.sub(r'[^\w\s가-힣.,!?%-/:()\[\]@=]', '', text)
    return text

cleaned_text = clean_special_chars(normalized_measurements)
print("최종 정리 결과:")
print(cleaned_text)


  실전 예제 4: 텍스트 전처리 파이프라인

📝 원본 텍스트
----------------------------------------------------------------------

환자ID: 2025-001234
주민번호: 850315-1******
연락처: 010-1234-5678
이메일: patient@hospital.com

특이사항:
- 고혈압   있음  (BP:  150/95)
- 당뇨    있음 (HbA1c: 7.2%)
- 알레르기: 페니실린, 아스피린

메모:  정기검진     필요!!!

URL: https://hospital.com/records/12345


📝 1. 개인정보 마스킹
----------------------------------------------------------------------
마스킹 결과:

환자ID: 2025-001234
주민번호: 850315-1******
연락처: 010-1234-****
이메일: pa***@hospital.com

특이사항:
- 고혈압   있음  (BP:  150/95)
- 당뇨    있음 (HbA1c: 7.2%)
- 알레르기: 페니실린, 아스피린

메모:  정기검진     필요!!!

URL: https://hospital.com/records/12345


📝 2. 공백 정규화
----------------------------------------------------------------------
공백 정규화 결과:
'\n환자ID: 2025-001234\n주민번호: 850315-1******\n연락처: 010-1234-****\n이메일: pa***@hospital.com\n\n특이사항:\n- 고혈압 있음 ('

📝 3. URL 추출 및 제거
----------------------------------------------------------------------
✓ 발견된 URL: ['https://hospital.com/records/12345']
URL 제거 

---

## 15. 정규표현식 성능 최적화

In [ ]:
print_section("정규표현식 성능 최적화")

print_example("1. 컴파일된 패턴 vs 문자열 패턴")

import time

test_text = "BP 120/80 mmHg " * 1000  # 반복 테스트용 텍스트

# 방법 1: 매번 컴파일
start = time.time()
for _ in range(100):
    re.findall(r'BP\s+\d+/\d+', test_text)
time_uncompiled = time.time() - start

# 방법 2: 사전 컴파일
pattern_compiled = re.compile(r'BP\s+\d+/\d+')
start = time.time()
for _ in range(100):
    pattern_compiled.findall(test_text)
time_compiled = time.time() - start

print(f"컴파일 없음: {time_uncompiled:.4f}초")
print(f"사전 컴파일: {time_compiled:.4f}초")
print(f"성능 향상: {time_uncompiled/time_compiled:.2f}배")

print_example("2. 탐욕적 vs 게으른 수량자 성능")

html_text = "<div>" + "content " * 100 + "</div>" * 10

# 탐욕적
start = time.time()
for _ in range(1000):
    re.findall(r'<div>.*?</div>', html_text)
time_lazy = time.time() - start

# 더 명확한 패턴
start = time.time()
for _ in range(1000):
    re.findall(r'<div>[^<]*</div>', html_text)
time_optimized = time.time() - start

print(f"게으른 수량자 (.*?): {time_lazy:.4f}초")
print(f"최적화된 패턴 ([^<]*): {time_optimized:.4f}초")
print(f"성능 향상: {time_lazy/time_optimized:.2f}배")

print_example("3. 백트래킹 피하기")

# 나쁜 예: 재앙적 백트래킹 가능
bad_pattern = r'(a+)+b'
good_pattern = r'a+b'

test_string = 'a' * 20 + 'c'  # 매칭 실패하는 경우

print("백트래킹 많은 패턴: (a+)+b")
print("최적화된 패턴: a+b")
print("→ 중첩된 수량자는 피해야 함!")


  정규표현식 성능 최적화

📝 1. 컴파일된 패턴 vs 문자열 패턴
----------------------------------------------------------------------
컴파일 없음: 0.0422초
사전 컴파일: 0.0432초
성능 향상: 0.98배

📝 2. 탐욕적 vs 게으른 수량자 성능
----------------------------------------------------------------------
게으른 수량자 (.*?): 0.0304초
최적화된 패턴 ([^<]*): 0.0033초
성능 향상: 9.26배

📝 3. 백트래킹 피하기
----------------------------------------------------------------------
백트래킹 많은 패턴: (a+)+b
최적화된 패턴: a+b
→ 중첩된 수량자는 피해야 함!


---

## 16. 실용적인 정규표현식 패턴 모음

In [ ]:
print_section("실용 패턴 라이브러리")

# 자주 사용하는 패턴들
patterns_library = {
    "이메일": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",
    "전화번호 (한국)": r"0\d{1,2}-?\d{3,4}-?\d{4}",
    "URL": r"https?://[^\s]+",
    "IPv4": r"\b(?:\d{1,3}\.){3}\d{1,3}\b",
    "날짜 (YYYY-MM-DD)": r"\d{4}-\d{2}-\d{2}",
    "시간 (HH:MM)": r"[0-2]\d:[0-5]\d",
    "한국 우편번호": r"\d{5}",
    "주민등록번호": r"\d{6}-[1-4]\d{6}",
    "혈압": r"\d{2,3}/\d{2,3}\s*mmHg",
    "체온": r"\d{2}\.\d\s*°C",
    "백분율": r"\d+(?:\.\d+)?%",
    "금액 (원)": r"\d{1,3}(?:,\d{3})*원",
    "HTML 태그": r"<[^>]+>",
    "16진수 컬러": r"#[0-9A-Fa-f]{6}",
}

print("자주 사용하는 정규표현식 패턴:\n")
for name, pattern in patterns_library.items():
    print(f"{name:20s}: {pattern}")

print_example("패턴 테스트")
test_samples = {
    "이메일": "contact@hospital.com",
    "전화번호 (한국)": "010-1234-5678",
    "혈압": "120/80 mmHg",
    "날짜 (YYYY-MM-DD)": "2025-04-16",
}

for pattern_name, sample_text in test_samples.items():
    pattern = patterns_library[pattern_name]
    match = re.search(pattern, sample_text)
    if match:
        print(f"✓ {pattern_name}: '{match.group()}'")


  실용 패턴 라이브러리
자주 사용하는 정규표현식 패턴:

이메일                 : [a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}
전화번호 (한국)           : 0\d{1,2}-?\d{3,4}-?\d{4}
URL                 : https?://[^\s]+
IPv4                : \b(?:\d{1,3}\.){3}\d{1,3}\b
날짜 (YYYY-MM-DD)     : \d{4}-\d{2}-\d{2}
시간 (HH:MM)          : [0-2]\d:[0-5]\d
한국 우편번호             : \d{5}
주민등록번호              : \d{6}-[1-4]\d{6}
혈압                  : \d{2,3}/\d{2,3}\s*mmHg
체온                  : \d{2}\.\d\s*°C
백분율                 : \d+(?:\.\d+)?%
금액 (원)              : \d{1,3}(?:,\d{3})*원
HTML 태그             : <[^>]+>
16진수 컬러             : #[0-9A-Fa-f]{6}

📝 패턴 테스트
----------------------------------------------------------------------
✓ 이메일: 'contact@hospital.com'
✓ 전화번호 (한국): '010-1234-5678'
✓ 혈압: '120/80 mmHg'
✓ 날짜 (YYYY-MM-DD): '2025-04-16'


---

## 17. 정규표현식 디버깅 및 테스팅

In [ ]:
print_section("정규표현식 디버깅")

print_example("1. 단계별 패턴 테스트")

debug_text = "혈압: 120/80 mmHg, 맥박: 72 bpm"

# 단계적으로 패턴 구축
patterns_steps = [
    (r"\d+", "숫자 찾기"),
    (r"\d+/\d+", "분수 형태 숫자"),
    (r"\d+/\d+\s*mmHg", "혈압 패턴"),
    (r"혈압:\s*(\d+/\d+\s*mmHg)", "라벨 포함"),
]

for pattern, description in patterns_steps:
    matches = re.findall(pattern, debug_text)
    print(f"\n패턴: {pattern}")
    print(f"설명: {description}")
    print(f"결과: {matches}")

print_example("2. 그룹 확인")

pattern_with_groups = r"(\w+):\s*(\d+/\d+|\d+)\s*(\w+)"
matches = list(re.finditer(pattern_with_groups, debug_text))

for i, match in enumerate(matches, 1):
    print(f"\n매칭 {i}:")
    print(f"  전체: {match.group(0)}")
    print(f"  그룹들: {match.groups()}")
    print(f"  그룹 1 (항목): {match.group(1)}")
    print(f"  그룹 2 (값): {match.group(2)}")
    print(f"  그룹 3 (단위): {match.group(3)}")

print_example("3. 매칭 실패 진단")

fail_text = "혈압 120-80"  # 잘못된 형식
fail_pattern = r"혈압:\s*\d+/\d+"

if not re.search(fail_pattern, fail_text):
    print(f"❌ 매칭 실패!")
    print(f"   텍스트: {fail_text}")
    print(f"   패턴: {fail_pattern}")
    print(f"\n💡 문제 진단:")
    print(f"   - 텍스트에 ':' 없음")
    print(f"   - 구분자가 '-'이지 '/'가 아님")
    print(f"\n수정된 패턴: r'혈압\\s*\\d+[-/]\\d+'")

    fixed_pattern = r"혈압\s*\d+[-/]\d+"
    if re.search(fixed_pattern, fail_text):
        print(f"✓ 수정된 패턴으로 매칭 성공!")


  정규표현식 디버깅

📝 1. 단계별 패턴 테스트
----------------------------------------------------------------------

패턴: \d+
설명: 숫자 찾기
결과: ['120', '80', '72']

패턴: \d+/\d+
설명: 분수 형태 숫자
결과: ['120/80']

패턴: \d+/\d+\s*mmHg
설명: 혈압 패턴
결과: ['120/80 mmHg']

패턴: 혈압:\s*(\d+/\d+\s*mmHg)
설명: 라벨 포함
결과: ['120/80 mmHg']

📝 2. 그룹 확인
----------------------------------------------------------------------

매칭 1:
  전체: 혈압: 120/80 mmHg
  그룹들: ('혈압', '120/80', 'mmHg')
  그룹 1 (항목): 혈압
  그룹 2 (값): 120/80
  그룹 3 (단위): mmHg

매칭 2:
  전체: 맥박: 72 bpm
  그룹들: ('맥박', '72', 'bpm')
  그룹 1 (항목): 맥박
  그룹 2 (값): 72
  그룹 3 (단위): bpm

📝 3. 매칭 실패 진단
----------------------------------------------------------------------
❌ 매칭 실패!
   텍스트: 혈압 120-80
   패턴: 혈압:\s*\d+/\d+

💡 문제 진단:
   - 텍스트에 ':' 없음
   - 구분자가 '-'이지 '/'가 아님

수정된 패턴: r'혈압\s*\d+[-/]\d+'
✓ 수정된 패턴으로 매칭 성공!


---

## 18. 실전 도전 과제

In [ ]:
print_section("실전 도전 과제")

print_example("과제 1: 복잡한 의료 처방전 파싱")

prescription = """
처방일: 2025-04-16
처방의: 김의사

[처방 내역]
1. 아스피린 100mg - 1일 1회, 아침 식후 30정
2. 메트포르민 500mg - 1일 2회, 아침/저녁 식후 60정
3. 리시노프릴 10mg - 1일 1회, 아침 공복 30정

[복약 지도]
- 아스피린: 위장장애 발생시 식사와 함께 복용
- 메트포르민: 저혈당 증상 주의
- 리시노프릴: 기침 발생시 의사와 상담

다음 진료: 2025-05-16
"""

print("📋 처방전:")
print(prescription)

print("\n🎯 과제: 다음 정보를 추출하세요")
print("   1. 약물명, 용량, 복용횟수, 복용시간, 처방일수")
print("   2. 복약 지도 내용")
print("   3. 다음 진료 날짜")

# 힌트: 약물 정보 추출 패턴
med_pattern = r"(\w+)\s+(\d+mg)\s+-\s+1일\s+(\d+)회,\s+([^0-9]+)\s+(\d+)정"
medications = re.findall(med_pattern, prescription)

print("\n✓ 추출 결과 (약물 정보):")
for med_name, dosage, frequency, timing, days in medications:
    print(f"   약물: {med_name}")
    print(f"   - 용량: {dosage}")
    print(f"   - 복용: 1일 {frequency}회, {timing}")
    print(f"   - 처방: {days}정")
    print()

print_example("과제 2: 로그 파일 분석")

log_data = """
2025-04-16 09:15:23 [INFO] User login: user@hospital.com
2025-04-16 09:15:45 [INFO] Record accessed: patient_12345
2025-04-16 09:16:12 [WARNING] Slow query detected: 2.5s
2025-04-16 09:17:03 [ERROR] Database connection failed: timeout
2025-04-16 09:17:15 [INFO] Retry successful
2025-04-16 09:18:30 [ERROR] File not found: /data/images/xray_001.jpg
"""

print("📋 로그 데이터:")
print(log_data)

print("\n🎯 과제: 다음 정보를 추출하세요")
print("   1. 모든 ERROR 레벨 로그")
print("   2. 타임스탬프와 로그 레벨별 분류")
print("   3. 에러 메시지 유형별 집계")

# 힌트: 로그 파싱 패턴
log_pattern = r"(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})\s+\[(\w+)\]\s+(.+)"
error_pattern = r"(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})\s+\[ERROR\]\s+(.+)"

errors = re.findall(error_pattern, log_data)
print(f"\n✓ ERROR 로그 {len(errors)}건:")
for timestamp, message in errors:
    print(f"   [{timestamp}] {message}")


  실전 도전 과제

📝 과제 1: 복잡한 의료 처방전 파싱
----------------------------------------------------------------------
📋 처방전:

처방일: 2025-04-16
처방의: 김의사

[처방 내역]
1. 아스피린 100mg - 1일 1회, 아침 식후 30정
2. 메트포르민 500mg - 1일 2회, 아침/저녁 식후 60정
3. 리시노프릴 10mg - 1일 1회, 아침 공복 30정

[복약 지도]
- 아스피린: 위장장애 발생시 식사와 함께 복용
- 메트포르민: 저혈당 증상 주의
- 리시노프릴: 기침 발생시 의사와 상담

다음 진료: 2025-05-16


🎯 과제: 다음 정보를 추출하세요
   1. 약물명, 용량, 복용횟수, 복용시간, 처방일수
   2. 복약 지도 내용
   3. 다음 진료 날짜

✓ 추출 결과 (약물 정보):
   약물: 아스피린
   - 용량: 100mg
   - 복용: 1일 1회, 아침 식후
   - 처방: 30정

   약물: 메트포르민
   - 용량: 500mg
   - 복용: 1일 2회, 아침/저녁 식후
   - 처방: 60정

   약물: 리시노프릴
   - 용량: 10mg
   - 복용: 1일 1회, 아침 공복
   - 처방: 30정


📝 과제 2: 로그 파일 분석
----------------------------------------------------------------------
📋 로그 데이터:

2025-04-16 09:15:23 [INFO] User login: user@hospital.com
2025-04-16 09:15:45 [INFO] Record accessed: patient_12345
2025-04-16 09:16:12 [WARNING] Slow query detected: 2.5s
2025-04-16 09:17:03 [ERROR] Database connection failed: timeout
2025-04-16 09:17:15 [IN

---

## 19. 정규표현식 도구 및 리소스

In [ ]:
print_section("유용한 도구 및 리소스")

tools_and_resources = """
🔧 온라인 테스트 도구:
   1. regex101.com - 가장 인기있는 정규표현식 테스터
      • 실시간 매칭 결과
      • 상세한 설명
      • 코드 생성 (Python, JavaScript 등)
      • URL: https://regex101.com

   2. regexr.com - 시각적 정규표현식 학습
      • 인터랙티브 인터페이스
      • 치트 시트 제공
      • URL: https://regexr.com

   3. regexper.com - 정규표현식 시각화
      • Railroad 다이어그램
      • 패턴 구조 이해에 유용
      • URL: https://regexper.com

📚 학습 리소스:
   1. Regular-Expressions.info
      • 포괄적인 튜토리얼
      • URL: https://www.regular-expressions.info

   2. RegexOne - 인터랙티브 튜토리얼
      • 단계별 연습 문제
      • URL: https://regexone.com

   3. Python re 모듈 문서
      • 공식 문서
      • URL: https://docs.python.org/3/library/re.html

🛠️ VSCode 확장:
   • Regex Previewer
   • Regex Explain
   • 검색창에서 정규표현식 사용: Ctrl+F → .* 아이콘 클릭

💡 유용한 팁:
   1. 항상 단순한 패턴부터 시작
   2. 온라인 도구로 먼저 테스트
   3. 주석을 활용한 VERBOSE 모드 사용
   4. 재사용 가능한 패턴은 변수로 저장
   5. 성능이 중요하면 패턴 컴파일
"""

print(tools_and_resources)


  유용한 도구 및 리소스

🔧 온라인 테스트 도구:
   1. regex101.com - 가장 인기있는 정규표현식 테스터
      • 실시간 매칭 결과
      • 상세한 설명
      • 코드 생성 (Python, JavaScript 등)
      • URL: https://regex101.com

   2. regexr.com - 시각적 정규표현식 학습
      • 인터랙티브 인터페이스
      • 치트 시트 제공
      • URL: https://regexr.com

   3. regexper.com - 정규표현식 시각화
      • Railroad 다이어그램
      • 패턴 구조 이해에 유용
      • URL: https://regexper.com

📚 학습 리소스:
   1. Regular-Expressions.info
      • 포괄적인 튜토리얼
      • URL: https://www.regular-expressions.info

   2. RegexOne - 인터랙티브 튜토리얼
      • 단계별 연습 문제
      • URL: https://regexone.com

   3. Python re 모듈 문서
      • 공식 문서
      • URL: https://docs.python.org/3/library/re.html

🛠️ VSCode 확장:
   • Regex Previewer
   • Regex Explain
   • 검색창에서 정규표현식 사용: Ctrl+F → .* 아이콘 클릭

💡 유용한 팁:
   1. 항상 단순한 패턴부터 시작
   2. 온라인 도구로 먼저 테스트
   3. 주석을 활용한 VERBOSE 모드 사용
   4. 재사용 가능한 패턴은 변수로 저장
   5. 성능이 중요하면 패턴 컴파일



---

## Summary

이 실습에서는 정규표현식(Regular Expression)의 기본 개념부터 고급 활용법까지 다양한 내용을 다루었습니다.

주요 학습 내용은 다음과 같습니다:

*   **기본 개념**: 정규표현식의 정의와 왜 필요한지에 대한 이론적 배경.
*   **앵커 (`^`, `$` )**: 문자열의 시작과 끝을 매칭하여 위치를 지정하는 방법.
*   **문자 클래스 (`[...]`)**: 특정 문자 집합 (`\d`, `\w`, `\s`)을 정의하거나 제외(`[^...]`)하여 매칭하는 방법.
*   **수량자 (`*`, `+`, `?`, `{m,n}`)**: 패턴의 반복 횟수를 지정하는 방법과 탐욕적/게으른 매칭의 차이점.
*   **그룹핑 (`(...)`)**: 패턴을 캡처하고 그룹화하는 방법, 이름있는 그룹(`(?P<name>...)`) 및 비캡처 그룹(`(?:...)`) 사용법.
*   **선택 (`|`)**: 여러 패턴 중 하나를 선택하여 매칭하는 방법.
*   **룩어라운드 (`(?=...)`, `(?<=...)`, `(?!...)`, `(?<!...)`)**: 실제 매칭에는 포함되지 않지만, 특정 맥락이 있는 경우에만 패턴을 매칭하는 고급 기법.
*   **플래그 (`(?i)`, `(?m)`, `(?s)`, `(?x)`)**: 정규표현식의 전역 동작 방식을 변경하는 옵션.
*   **실전 예제**: 의료 텍스트 파싱, 데이터 검증 및 정규화, 텍스트 클렌징 등 실제 데이터 처리 시나리오에서 정규표현식을 활용하는 방법.
*   **성능 최적화**: 컴파일된 패턴, 게으른 수량자, 백트래킹 방지 등 성능을 고려한 정규표현식 작성 팁.
*   **디버깅 및 테스트**: 패턴을 단계별로 구축하고, 그룹을 확인하며, 매칭 실패 시 진단하는 방법.
*   **유용한 도구 및 리소스**: 정규표현식 학습 및 테스트에 도움이 되는 온라인 도구와 학습 자료를 소개했습니다.

정규표현식은 텍스트 데이터를 다루는 데 있어 매우 강력하고 유연한 도구이며, 이 실습을 통해 그 잠재력을 충분히 이해하고 활용할 수 있는 기반을 다지셨기를 바랍니다.